In [ ]:
# Make sure we are pointing to the directory that has all the files necessary
import numpy as np 
import pandas as pd 
import sys, requests, shutil, os

In [ ]:
cd "/content/sample_data/dataset2"

/content/sample_data/dataset2


In [ ]:
pwd

'/content/drive/My Drive/ADM_2/dataset2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/ADM_2/dataset2.zip" -d "/content/sample_data"

Streaming output truncated to the last 5000 lines.
  inflating: /content/sample_data/dataset2/validation_images_model/6542/fedb11dfe8669d4f.jpg  
  inflating: /content/sample_data/dataset2/validation_images_model/6542/ff564805490e845e.jpg  
   creating: /content/sample_data/dataset2/validation_images_model/6543/
  inflating: /content/sample_data/dataset2/validation_images_model/6543/6f7a69a5bc6d40ae.jpg  
   creating: /content/sample_data/dataset2/validation_images_model/6544/
  inflating: /content/sample_data/dataset2/validation_images_model/6544/09d33fbbd531dd97.jpg  
  inflating: /content/sample_data/dataset2/validation_images_model/6544/2201eff9af5c5173.jpg  
  inflating: /content/sample_data/dataset2/validation_images_model/6544/4d1a47e26bded942.jpg  
  inflating: /content/sample_data/dataset2/validation_images_model/6544/5696ea3defda98cf.jpg  
  inflating: /content/sample_data/dataset2/validation_images_model/6544/570b9048c3422bc0.jpg  
  inflating: /content/sample_data/dataset2/

In [ ]:
import numpy as np 
import pandas as pd 
import sys, requests, shutil, os

data_train=pd.read_csv("train_sample.csv")
data_valid=pd.read_csv("valid_sample.csv")
data_test=pd.read_csv("test_sample.csv")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
#from tensorflow.keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
import csv
import os
#import cv2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import math
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import tensorflow as tf
import tensorflow.keras

In [ ]:
# This cell counts the total number of train images and validation images in the entire dataset
# It stores the counts of train data 108181 training samples belonging to 1000 classes and 
# 19826 test samples in the respective variables nb_train_samples and nb_validation_samples



train_data_dir = 'train_images_model'                   # points to directory having train images
validation_data_dir = 'validation_images_model'         # points to directory having validation images


def count(dir):
    i = 5000
    count = []
    while i <= 6999:
        f = str(i)
        #print (f)
        for root, dirs, files in os.walk(dir +'/'+ f):  # os. walk() is used to iterate through all folders of a directory
            for pic in files:
                count.append(f)   
            i += 1
    print (len(count))
    return ([len(count),count])

nb_train_samples = count(train_data_dir)
nb_validation_samples = count(validation_data_dir)


63750
19380


In [ ]:
#Importing few other files needed
import pandas as pd
import os
import shutil
from shutil import copyfile
import urllib

In [ ]:
# here we build the VGG 16 model, which is pretrained using the imagenet dataset,
# This is achieved by setting weights = 'imagenet' as a parameter while building the model

img_width, img_height = 96, 96 #dimensions used in the model
top_model_weights_path = 'bottleneck_fc_model.h5' # this is used to save weights in later stages, avoid recomputations
epochs = 5
batch_size = 510 #GCD
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=30,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range = 0.5,
                                 brightness_range = [0.5,1.5])

    # VGG 16 FramerWork
    model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(96,96,3))
    print ('start1')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, 
        shuffle=False)
    
    print ('start2')
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples[0] // batch_size) 
    print ('bottleneck_features_trained')
    
    
    with open('bottleneck_features_train.npy', 'wb') as features_train_file:
        np.save(features_train_file, bottleneck_features_train)
    print ('Train done')
    

    datagen = ImageDataGenerator(rescale=1. / 255) 
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print ('validation predict start')
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples[0] // batch_size)
    
    with open('bottleneck_features_validation.npy', 'wb') as features_validation_file:
        np.save(features_validation_file, bottleneck_features_validation)
    print ('validation done')
save_bottleneck_features()

start1
Found 63750 images belonging to 2000 classes.
start2
Instructions for updating:
Please use Model.predict, which supports generators.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


bottleneck_features_trained
Train done
Found 19380 images belonging to 2000 classes.
validation predict start
validation done


In [ ]:
# counts of train and validation labels
train_labels = np.array(nb_train_samples[1])
train_labels = [str(int(train_label) - 5000) for train_label in train_labels] 
print(len(train_labels))
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
print(len(train_data))
validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
print(len(validation_data))
validation_labels = np.array(nb_validation_samples[1])
print(len(validation_labels))

63920
63920
19380
19380


In [ ]:
epochs = 5
batch_size = 510
import numpy as np

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(nb_train_samples[1])
    train_labels = [str(int(train_label) - 5000) for train_label in train_labels] 
    # we do label-1000, so that class lable starts from 0, as we have labels starting from 1000

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(nb_validation_samples[1])
    validation_labels = [str(int(validation_label) - 5000) for validation_label in validation_labels]
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    n_class = 2000 # max. classes given to the model
    model.add(Dense(n_class, activation='softmax'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    train_labels = to_categorical(train_labels, n_class)
    validation_labels = to_categorical(validation_labels, n_class)
    
    print ('model fit starting')
    
    model.fit(train_data, train_labels,epochs=epochs,batch_size=batch_size,validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

train_top_model()

model fit starting
Epoch 1/5
125/125 [==============================] - 2s 15ms/step - loss: 5.7377 - accuracy: 0.0910 - val_loss: 4.7274 - val_accuracy: 0.1746
Epoch 2/5
125/125 [==============================] - 2s 14ms/step - loss: 4.7379 - accuracy: 0.1807 - val_loss: 4.0420 - val_accuracy: 0.2614
Epoch 3/5
125/125 [==============================] - 2s 13ms/step - loss: 4.1611 - accuracy: 0.2398 - val_loss: 3.5902 - val_accuracy: 0.3257
Epoch 4/5
125/125 [==============================] - 2s 13ms/step - loss: 3.7362 - accuracy: 0.2842 - val_loss: 3.3032 - val_accuracy: 0.3641
Epoch 5/5
125/125 [==============================] - 2s 13ms/step - loss: 3.4046 - accuracy: 0.3229 - val_loss: 3.2014 - val_accuracy: 0.3829


In [ ]:
base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(96,96,3))
print(base_model.input)

Tensor("input_12:0", shape=(None, 96, 96, 3), dtype=float32)


In [ ]:
pwd

'/content/drive/My Drive/ADM_2/dataset2'

In [ ]:
# This is where we fine tune the pretrained model according to our dataset

img_width, img_height = 96, 96
top_model_weights_path = 'bottleneck_fc_model.h5' 
train_data_dir = 'train_images_model'
validation_data_dir = 'validation_images_model'
batch_size = 200
epochs = 50
def trainCNN():

    # build the VGG16 network

    base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(96,96,3))

    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(256, activation='relu'))
    n_class = 2000
    top_model.add(Dense(n_class, activation='softmax'))
    top_model.load_weights(top_model_weights_path) #Load the weights initialized in previous steps
    

    model = Model(base_model.input, top_model(base_model.output))
    
    # set the first 16 layers to non-trainable (weights will not be updated) 
    # 1 conv layer and three dense layers will be trained
    for layer in model.layers[:16]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0.9,beta_2=0.999,epsilon=1e-8, decay=0.0),
                  metrics=['accuracy'])
    print ('Compilation done.')
    
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=90,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range = 0.5)
    
    valid_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    np.save('class_indices.npy', train_generator.class_indices)

    validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    
    print ('Model fit begins...')
    history=model.fit_generator(
        train_generator,
        steps_per_epoch=150,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=75,
        callbacks=[ModelCheckpoint(filepath=top_model_weights_path, save_best_only=True, save_weights_only=True)]
        )
    
    model.save_weights(top_model_weights_path)
    # final weights are saved in bottleneck_fc_model.h5 file

trainCNN()

Compilation done.
Found 63750 images belonging to 2000 classes.
Found 19380 images belonging to 2000 classes.
Model fit begins...


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1/50
150/150 [==============================] - 148s 985ms/step - loss: 5.9212 - accuracy: 0.0644 - val_loss: 4.8089 - val_accuracy: 0.1310
Epoch 2/50
150/150 [==============================] - 143s 956ms/step - loss: 4.8514 - accuracy: 0.1346 - val_loss: 4.2770 - val_accuracy: 0.1929
Epoch 3/50
150/150 [==============================] - 142s 948ms/step - loss: 4.5148 - accuracy: 0.1729 - val_loss: 3.9892 - val_accuracy: 0.2443
Epoch 4/50
150/150 [==============================] - 139s 927ms/step - loss: 4.2142 - accuracy: 0.2106 - val_loss: 3.7237 - val_accuracy: 0.2791
Epoch 5/50
150/150 [==============================] - 140s 931ms/step - loss: 4.0307 - accuracy: 0.2354 - val_loss: 3.5047 - val_accuracy: 0.3213
Epoch 6/50
150/150 [==============================] - 146s 975ms/step - loss: 3.8505 - accuracy: 0.2577 - val_loss: 3.4891 - val_accuracy: 0.3185
Epoch 7/50
150/150 [==============================] - 142s 949ms/step - loss: 3.7308 - accuracy: 0.2775 - val_loss: 3.2901 -

In [ ]:
from tensorflow.python.platform import app
import argparse
import os
import sys
import time
from time import *
import io
import tensorflow as tf 
import cv2

In [ ]:
top_model_weights_path = 'bottleneck_fc_model.h5'       # final weights of the model
train_data_dir = 'train_images_model'
testfile = 'test_images_from_train'
subfile = 'result2.csv'                                  # predictions of the model

In [ ]:
from keras import backend as K 
import cv2

# here we send the test images to the model
# the model predicts the classes with some confidence score
# these tuples are stored in a file result.csv that are used in later parts of the code

def predict(image_path):
    print ('starting...')
    path, dirs, files = next(os.walk(image_path))
    file_len = len(files)
    print('Number of Testimages:', file_len)

    train_datagen = ImageDataGenerator(rescale=1. / 255)

    generator = train_datagen.flow_from_directory(train_data_dir, batch_size=batch_size)
    label_map = (generator.class_indices)

    n_class = 2000

    with open(subfile, 'w') as csvfile:
        newFileWriter = csv.writer(csvfile)
        newFileWriter.writerow(['id', 'landmarks'])

        file_counter = 0
        base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

        top_model = Sequential()
        top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
        top_model.add(Dense(256, activation='relu'))
        top_model.add(Dense(256, activation='relu'))
        top_model.add(Dense(n_class, activation='softmax'))

        model = Model(base_model.input, top_model(base_model.output))
        model.load_weights(top_model_weights_path)
        for root, dirs, files in os.walk(image_path): 
            for pic in files:

                #loop folder and convert image
                path = image_path + '/' + pic


                orig = cv2.imread(path)
                image = load_img(path, target_size=(96, 96))
                image = img_to_array(image)

                image = image / 255

                image = np.expand_dims(image, axis=0)

                #classify landmark
                

                prediction = model.predict(image)

                class_predicted = prediction.argmax(axis=1)
                inID = class_predicted[0]
                inv_map = {v: k for k, v in label_map.items()}
                label = inv_map[inID]

                score = max(prediction[0])
                scor = "{:.2f}".format(score)
                out = str(label) + ' '+ scor
                #print(score)

                newFileWriter.writerow([os.path.splitext(pic)[0], out])
                

predict(testfile)

In [ ]:
from keras import backend as K 
import cv2
top_model_weights_path = 'bottleneck_fc_model.h5'       # final weights of the model
train_data_dir = 'train_images_model'
testfile = 'test_images_from_train'
subfile = 'result2.csv'                                  # predictions of the model
def predict(image_path):
    print ('starting')
    path, dirs, files = next(os.walk(image_path))
    file_len = len(files)
    print('Number of Testimages:', file_len)
    #train_datagen = ImageDataGenerator(rescale=1. / 255)
    #generator = train_datagen.flow_from_directory(train_data_dir, batch_size=batch_size)
    #label_map = (generator.class_indices)
    label_map={}
    for i in range(2000):
      label_map[str(5000+i)]=i
    n_class = 2000
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dense(n_class, activation='softmax'))
    model = Model(base_model.input,top_model(base_model.output))
    model.load_weights(top_model_weights_path)
    with open(subfile, 'w') as csvfile:
        newFileWriter = csv.writer(csvfile)
        newFileWriter.writerow(['id', 'landmarks','confidence'])
        file_counter = 0
        for root, dirs, files in os.walk(image_path):  # loop through startfolders
            i=0
            for pic in files:
                i+=1
                #loop folder and convert image
                path = image_path + '/' + pic


                orig = cv2.imread(path)
                image = load_img(path, target_size=(96, 96))
                image = img_to_array(image)

                # important! otherwise the predictions will be '0'
                image = image / 255

                image = np.expand_dims(image, axis=0)

                #classify landmark
                

                prediction = model.predict(image)


                class_predicted = prediction.argmax(axis=1)
                #class_predicted = np.argmax(prediction,axis=1)
                #print (pic, class_predicted)


                inID = class_predicted[0]
                #print inID

                inv_map = {v: k for k, v in label_map.items()}
                #print class_dictionary
                label = inv_map[inID]
                score = max(prediction[0])
                scor = "{:.2f}".format(score)
                #out = str(label) + ' '+ scor
                print (i,score,label)
                newFileWriter.writerow([os.path.splitext(pic)[0], int(label),scor])

predict(testfile)

starting
Number of Testimages: 4783
1 0.42045176 5525
2 0.159241 6743
3 0.9933908 5203
4 0.17908786 6043
5 0.18792956 6039
6 0.25939888 6696


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


7 0.15413055 5150
8 0.20745432 5064
9 0.24745308 5873
10 0.24442485 6665
11 0.99918026 5046
12 0.6716029 6399
13 0.090346955 6464
14 0.7713827 6594
15 0.4684247 5590
16 0.36062244 6749
17 0.18792956 6039
18 0.29119602 5574
19 0.53271925 5421
20 0.5562708 6349
21 0.8518459 6599
22 0.7944022 6130
23 0.19865018 6582
24 0.18792956 6039
25 0.70509815 6259
26 0.99207956 5101
27 0.695005 5887
28 0.26715055 5590
29 0.40882507 6213
30 0.18879946 6125
31 0.87046033 6321
32 0.9069552 6597
33 0.25939888 6696
34 0.2477493 6597
35 0.5781316 5350
36 0.36862627 6051
37 0.99638087 5376
38 0.64623046 6696
39 0.9918154 5943
40 0.761157 6231
41 0.80054253 6464
42 0.58647174 6876
43 0.9991547 6991
44 0.26218653 6511
45 1.0 5367
46 0.7337774 6696
47 0.9935493 5046
48 0.9558651 5206
49 0.3590184 5210
50 0.99957746 6685
51 0.7841768 6582
52 0.18003227 6266
53 0.18792956 6039
54 0.55688775 6476
55 0.4170189 5421
56 0.9206553 6066
57 0.99084455 5986
58 0.77318364 5919
59 0.17442687 5887
60 0.600709 5955
61 0.52

In [ ]:
result = pd.read_csv('result2.csv')     # predicted values

In [ ]:
test = pd.read_csv('test_sample.csv')  # ground truth

In [ ]:
# final accuracy of the model as tested on test dataset
import pandas as pd
def test_accuracy(pred,truth):
    result=pd.read_csv(pred)
    #print(result.head(20))
    test=pd.read_csv(truth)
    count=0
    for i in result["id"]:
        a=result.loc[result['id'] == i, "landmarks"]
        b=a.values
        #b=list(b[0].split(" "))
        b[0]=int(b[0])
        if i in test.id.values:
          c=test.loc[test['id'] == i, "landmark_id"]
          d=c.values
          if b[0]==d[0]:
              count+=1
    print(count/len(result))
    return count/len(result)
test_accuracy("result2.csv","test_sample.csv")

0.5410830022998119


0.5410830022998119

In [ ]:
# This call calulates the Global Average Precision Metric 
# 
import pandas as pd
def GAP_metric(path):
    result=pd.read_csv(path)
    test=pd.read_csv("test_sample.csv")
    result=result.sort_values(by='confidence', ascending=False)
    M=0
    true=0
    total=0
    s=0
    relevance=0
    for i in result['id']:
        total+=1
        a=result.loc[result['id'] == i, "landmarks"]
        b=a.values
        #if i in test['id']:
        if i in test.id.values:
          M+=1
          c=test.loc[test['id'] == i, "landmark_id"]
          d=c.values
          if b[0]==d[0]:
              true+=1
              relevance=1
        s+=(true/total)*relevance
        relevance=0
    print(M,s,total,true)
    print(s/M)
    return s/M
GAP_metric("result.csv")

In [ ]:
GAP_metric('result2.csv')

4783 2345.7565041894886 4783 2588
0.4904362333659813


0.4904362333659813